In [4]:
import geopandas as gpd
import fiona
import folium as fm
import pandas as pd
import numpy as np

import os, sys
import glob
from tqdm import tqdm

from pyproj import Proj, Transformer, transform
from shapely.geometry import Polygon
# 위도 경도까지 찍어서 DB에 넣어줘야함
# 토지 고유번호 맞추는 부분은 표제부 쪽 데이터 점검
# 폴리곤의 토지고유번호 맞추는 것도 원본 확인해봐야함

# (11.5~ EDA 후반 시작)
# 도형정보의 토지고유번호를 맞추는 것 1번 V
# 토지고유번호를 맞췄으면 폴리곤에 위도 경도까지 변환해서 입히는 것 2번
# 그걸 컬럼별로 나눠서 클릭했을 때 그 토지고유번호에서 실거래랑 도형정보까지 프론트에서 보여주는게 3번
# 거기에 표제부를 얹어서 건물정보까지 나오는게 4번 (노후화)

In [2]:
# 건물도형_전체분_서울특별시_2023_9월 데이터 점검
a = gpd.read_file("./data/건물도형_전체분_서울특별시_2023_9월/Total.JUSURB.20231001.TL_SGCO_RNADR_MST.11000.shp",encoding='cp949')
a_ = gpd.read_file("./data/건물도형_전체분_서울특별시_2023_9월/Total.JUSURB.20231001.TL_SPBD_ENTRC.11000.shp",encoding='cp949')
a__ = gpd.read_file("./data/건물도형_전체분_서울특별시_2023_9월/Total.JUSURB.20231001.TL_SPOT_CNTC.11000.shp",encoding='cp949')
b = gpd.read_file("data/도로명주소_전자지도_2023_9월/11000/TL_SPBD_BULD.shp", encoding = 'cp949')

In [5]:
a.sample(1)
# a.columns

,ADR_MNG_NO,SIG_CD,RN_CD,BULD_SE_CD,BULD_MNNM,BULD_SLNO,BUL_MAN_NO,EQB_MAN_SN,EFFECT_DE,geometry
82975,11170128410641300002600000,11170,4106413,0,26,0,16543,0,20110729,"POLYGON ((952598.152 1947635.868, 952593.103 1..."


In [25]:
b.sample(5)

,BDTYP_CD,BD_MGT_SN,BSI_INT_SN,BSI_ZON_NO,BULD_MNNM,BULD_NM,BULD_NM_DC,BULD_SE_CD,BULD_SLNO,BUL_DPN_SE,...,MVM_RES_CD,NTFC_DE,OPERT_DE,POS_BUL_NM,RDS_MAN_NO,RDS_SIG_CD,RN_CD,SIG_CD,UND_FLO_CO,geometry
492220,01001,1165010800115190012021239,23576,06668,39,None,None,0,0,M,...,71,20110729,None,None,1486,11650,4163184,11650,1,"POLYGON ((956339.643 1943004.209, 956337.285 1..."
454796,01000,1162010100109260018039969,15164,08757,388,None,None,0,7,M,...,71,20110729,20131203112205,None,1675,11620,3005083,11620,0,"POLYGON ((950772.481 1942756.977, 950776.335 1..."
324068,02003,1141011900100030126000001,30018,03669,24,동화 빌라,None,0,5,M,...,71,20110729,None,동화빌라,3232,11410,4136345,11410,1,"POLYGON ((948434.271 1953779.263, 948423.043 1..."
440437,04402,1159010300107590000029278,16632,06968,262,None,None,0,0,M,...,76,20110729,20201104104412,None,1162,11590,3005080,11590,1,"POLYGON ((951241.160 1945021.705, 951232.455 1..."
592768,16001,1174010900102800008013661,25196,05330,258,None,None,0,6,M,...,51,20161214,20191206163414,None,1535,11740,3016054,11740,0,"POLYGON ((967513.436 1949361.574, 967509.182 1..."


In [35]:
b.columns

b.sample(5)[["SIG_CD","EMD_CD","LI_CD","MNTN_YN","LNBR_MNNM","LNBR_SLNO","BULD_NM"]] 
# 시군구 읍면동 리 산여부 본번 부번 (5 + 5 + 1 + 4 + 4) = 19
# 미성아파트를 예시로 11530 + 101 + 00 + 산(1) + 290번지(0290) + 0번지(0000) = 1153010100102900000으로 실거래 가 (t)랑 비교하면?

,SIG_CD,EMD_CD,LI_CD,MNTN_YN,LNBR_MNNM,LNBR_SLNO,BULD_NM
414148,11530,101,00,0,290,0,미성아파트
302844,11410,117,00,0,120,7,None
460577,11620,101,00,0,1686,11,None
361200,11500,109,00,0,880,0,치현마을동일스위트리버아파트
347576,11110,101,00,0,52,134,None


In [36]:
rp = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",encoding='cp949')

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_60609/4255262594.py:1: DtypeWarning: Columns (11,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  rp = pd.read_csv("./data/실거래_가공/아파트_실거래_10년_가공.csv",encoding='cp949')


In [42]:
rp.columns

Index(['Unnamed: 0', '시군구', '번지', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)',
       '계약년월', '계약일', '보증금(만원)', '월세(만원)', '층', '건축년도', '도로명', '계약기간', '계약구분',
       '갱신요구권 사용', '종전계약 보증금 (만원)', '종전계약 월세 (만원)', '법정동코드', '산', '토지고유번호'],
      dtype='object')

In [47]:
t = rp[rp['토지고유번호'] == 1153010100102900000]
t[t['계약년월'] == 202309][['전월세구분', '전용면적(㎡)','계약년월', '계약일', '보증금(만원)', '월세(만원)','계약구분','갱신요구권 사용','층']]
# 맞게 조회가 되는 것을 확인 -> 따라서 해당 데이터 프레임으로 고정하면 건물도형은 실거래와 조인가능! 
# 건축물 정보를 이제 조인할 차례(다음 번 차례)

,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),계약구분,갱신요구권 사용,층
1836279,전세,52.45,202309,1,"26,500",0,-,-,3.0
1836280,월세,83.67,202309,2,"7,000",110,-,-,8.0
1836281,월세,37.91,202309,2,"2,000",60,신규,-,8.0
1836282,전세,66.03,202309,8,"29,000",0,-,-,13.0
1836283,전세,66.03,202309,8,"29,000",0,-,-,13.0
1836284,전세,37.91,202309,8,"19,950",0,-,-,7.0
1836285,전세,66.03,202309,8,"29,000",0,신규,-,13.0
1836286,전세,37.91,202309,11,"15,000",0,신규,-,8.0
1836287,월세,37.91,202309,13,"11,000",35,갱신,-,10.0
1836288,월세,52.45,202309,13,"6,000",65,갱신,-,1.0


In [34]:
test = b.sample(10)[["BD_MGT_SN","BULD_MNNM","BULD_SLNO","BULD_NM"]]
test["BD_MGT_SN"] = test["BD_MGT_SN"].str[:-6]

In [27]:
b.iloc[518555]

BDTYP_CD                                                  02001
BD_MGT_SN                             1168011300100770000000005
BSI_INT_SN                                                27641
BSI_ZON_NO                                                06370
BULD_MNNM                                                   260
BULD_NM                                                 강남한양수자인
BULD_NM_DC                                                 422동
BULD_SE_CD                                                    0
BULD_SLNO                                                     0
BUL_DPN_SE                                                    M
BUL_ENG_NM                                                 None
BUL_MAN_NO                                                26111
EMD_CD                                                      112
EQB_MAN_SN                                                  982
GRO_FLO_CO                                                   10
LI_CD                                   

In [23]:
test

1168011200106860000

,BD_MGT_SN,BULD_MNNM,BULD_SLNO,BULD_NM
421510,1156011800101300027,9,0,None
548161,1174010900103260006,7,16,None
298587,1141011600102950000,14,0,None
163635,1129013300104130013,16,2,None
518555,1168011300100770000,260,0,강남한양수자인
275599,1138010400100120501,4,4,None
408567,1153010200111290009,76,30,None
427381,1154510300102140012,53,2,None
306645,1141010100100890000,71,1,None
394054,1153010200100820007,232,11,None


In [ ]:


# GeoDataFrame 생성
data = {'geometry': [Point(0, 0), Point(1, 1), Point(2, 2)]}
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")

# Polygon 객체의 위도 경도 변환 (Shapely 사용)
gdf['geometry'] = gdf['geometry'].apply(lambda point: point.buffer(1.0))  # 예시로 1.0의 버퍼를 추가함

# Folium 맵 생성
m = folium.Map(location=[0, 0], zoom_start=10)

# GeoDataFrame을 GeoJSON 형식으로 변환 (Fiona 사용)
gdf.to_file("polygons.geojson", driver='GeoJSON')

# GeoJSON 파일을 Folium 맵에 추가
folium.GeoJson("polygons.geojson").add_to(m)

# Folium 맵을 HTML 파일로 저장
m.save("map.html")


In [15]:
test = b[['BD_MGT_SN','BSI_ZON_NO','LNBR_MNNM','LNBR_SLNO',"EMD_CD"]].sample(5)

In [16]:
test

,BD_MGT_SN,BSI_ZON_NO,LNBR_MNNM,LNBR_SLNO,EMD_CD
66394,1117010700100240006024364,04334,24,6,107
483051,1162010200104710029014361,08704,471,29,102
132649,1123011000103320008003854,02451,332,8,110
447602,1156010300102350001034448,07253,235,0,103
473051,1162010200100920248022502,08783,92,246,102


In [19]:
test["BD_MGT_SN"]

/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_49245/3254293256.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  test["BD_MGT_SN"][5:3]


Series([], Name: BD_MGT_SN, dtype: object)